<a href="https://colab.research.google.com/github/ort-eila/git_kundaje_annotations/blob/main/step_1_kb_from_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# kb from wikipedia

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import os
import pandas as pd
import string  # Import the string module

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup
import os
import pandas as pd

def extract_text_from_wikipedia(url, txt_filename):
    try:
        res = requests.get(url)
        res.raise_for_status()
        wiki = bs4.BeautifulSoup(res.text, "html.parser")

        # Extract all paragraphs into a list
        paragraphs = [paragraph.getText() for paragraph in wiki.select('p')]

        # Join the paragraphs into a single text with a newline character at the end
        text = ''.join(paragraphs)
        print("text is ",text)
        text = text.replace("\n", "")

        # Open a file named after the Wikipedia page in write mode
        with open(txt_filename, "w", encoding="utf-8") as f:
            f.write(text) # + '\n')

        print(f"Extracted text from '{url}' and saved to '{txt_filename}'.")
    except Exception as e:
        print(f"Error extracting text: {str(e)}")


In [4]:
def update_csv_with_wikipedia_details(url_list, csv_filename, remove_wiki_page_list=None):
    if os.path.exists(csv_filename):
        # Read the existing CSV into a DataFrame with headers
        existing_df = pd.read_csv(csv_filename)
    else:
        # Create an empty DataFrame with headers if the CSV doesn't exist
        existing_df = pd.DataFrame(columns=['title', 'text'])

    # Create a set of existing titles for fast lookup
    existing_titles = set(existing_df['title'])

    # Create an empty list to store DataFrames
    df_list = []

    for wiki_page in url_list:
        if wiki_page in existing_titles:
            print(f"Skipping '{wiki_page}' as it already exists in the CSV.")
            continue

        # Define the Wikipedia page URL
        url = f'https://en.wikipedia.org/wiki/{wiki_page}'

        # Define the filename for the text file
        txt_filename = f"{wiki_page}.txt"

        # Call the extract_text_from_wikipedia method to save text to a file
        extract_text_from_wikipedia(url, txt_filename)

        # Check if the text file was successfully generated
        if os.path.exists(txt_filename):
            # Read the content from the text file
            with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                text = txt_file.read()

            # Create a DataFrame with the title and text
            df = pd.DataFrame({'title': [wiki_page], 'text': [text]})

            # Append the DataFrame to the list
            df_list.append(df)

            # Remove the generated text file
            #DEBUG
            # os.remove(txt_filename)

            print(f"Downloaded '{wiki_page}' and created DataFrame.")
        else:
            print(f"Failed to download '{wiki_page}'.")

    if df_list:
        # Concatenate all DataFrames in the list into one
        final_df = pd.concat(df_list, ignore_index=True)

        if os.path.exists(csv_filename):
            # Append the final DataFrame to the existing CSV file without writing the header
            final_df.to_csv(csv_filename, mode='a', header=False, index=False)
        else:
            # Write the final DataFrame with the header if the CSV doesn't exist
            final_df.to_csv(csv_filename, mode='w', header=True, index=False)

        print(f"Updated CSV file '{csv_filename}' with missing Wikipedia page details.")
    else:
        print("No new data to update in the CSV.")

    # Remove entries from the CSV based on remove_wiki_page_list
    if remove_wiki_page_list:
        existing_df = existing_df[~existing_df['title'].isin(remove_wiki_page_list)]
        existing_df.to_csv(csv_filename, index=False)

        print(f"Removed specified entries from CSV.")


In [5]:
# Example usage with a list of wiki_page names to update and remove
wiki_page_list = [
    "BRCA_mutation",
    "BRCA1",
    "BRCA2"]
    # "Tumor_suppressor_genes",
    # "Adenomatous_polyposis_coli",
    # "ATOH1",
    # "BCL10",
    # "Cadherin-1",
    # "Capicua_(protein)",
    # "CDKN1B"]
#     "CHEK2",
#     "Cyclin-dependent_kinase_inhibitor_1C",
#     "DHX15",
#     "DLD/NP1",
#     "Protein"
#     # Add more wiki_page names as needed
# ]

remove_wiki_page_list = []
    # "BRCA1",
    # "BRCA2",
    # "Tumor_suppressor_genes",
    # "Adenomatous_polyposis_coli",
    # "ATOH1",
    # "BCL10",
    # "Cadherin-1",
    # "Capicua_(protein)",
    # "CDKN1B"]
    # Add more page titles to remove as needed
# ]

update_csv_with_wikipedia_details(wiki_page_list, "wikipedia_details.csv", remove_wiki_page_list)


text is  
A BRCA mutation is a mutation in either of the BRCA1 and BRCA2 genes, which are tumour suppressor genes. Hundreds of different types of mutations in these genes have been identified, some of which have been determined to be harmful, while others have no proven impact. Harmful mutations in these genes may produce a hereditary breast–ovarian cancer syndrome in affected persons. Only 5–10% of breast cancer cases in women are attributed to BRCA1 and BRCA2 mutations (with BRCA1 mutations being slightly more common than BRCA2 mutations), but the impact on women with the gene mutation is more profound.[2] Women with harmful mutations in either BRCA1 or BRCA2 have a risk of breast cancer that is about five times the normal risk, and a risk of ovarian cancer that is about ten to thirty times normal.[3] The risk of breast and ovarian cancer is higher for women with a high-risk BRCA1 mutation than with a BRCA2 mutation. Having a high-risk mutation does not guarantee that the woman will 

In [7]:
!ls


BRCA1.txt  BRCA2.txt  BRCA_mutation.txt  sample_data  wikipedia_details.csv


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
# !ls /content/drive/MyDrive

In [15]:
import shutil

# Define the source file path
source_file = 'wikipedia_details.csv'

# Define the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/bio-llm/'

# Copy the file to Google Drive
shutil.copy(source_file, destination_folder)


'/content/drive/MyDrive/bio-llm/wikipedia_details.csv'